# Hypothesis testing on LSNs agreemnent values

> **Analysis type**: Single γ (results)

In [ ]:
import json
from os.path import join

import numpy as np
import pandas as pd
from dn_utils.networks import fdrcorrection_matrix
from dn_utils.path import path
from dn_utils.plotting import plot_matrix

%matplotlib inline

In [ ]:
atlas = "combined_roi_4and5"
gamma = 2

In [ ]:
# Create additional paths
gamma_str = str(float(gamma)).replace('.', '_')
path_corrmats = join(path["bsc"], "corrmats")
path_corrmats_unthr = join(path_corrmats, atlas, "unthr")
path_corrmats_unthr_gamma = join(path_corrmats_unthr, f"gamma_{gamma_str}")

# Load subject exclusion
df_exclusion = pd.read_csv(join(path["nistats"], "exclusion/exclusion.csv"), 
                           index_col=0)
ok_index = df_exclusion["ok_all"]

# Meta information about corrmats dimensions
with open(join(path_corrmats, atlas, "corrmats_aggregated.json"), "r") as f:
    corrmats_meta = json.loads(f.read()) 
    
# Load ROI information
df_roi = pd.read_csv(
    join(path_corrmats, atlas, "roi_table_filtered.csv"), index_col=0)
df_roi = df_roi.reset_index()
network_names = df_roi["netName"].unique()
    
# Load data
d = np.load(join(path_corrmats_unthr_gamma, "d_networks.npy"))    
d_null = np.load(join(path_corrmats_unthr_gamma, "d_networks_null.npy"))
pvals = np.load(join(path_corrmats_unthr_gamma, "d_pvals.npy"))
fstats = np.load(join(path_corrmats_unthr_gamma, "d_fstats.npy"))
fstats_null = np.load(join(path_corrmats_unthr_gamma, "d_fstats_null.npy"))

d = d[ok_index]
d_null = d_null[:, ok_index]

n_nets = len(network_names)
n_reps = len(d_null)
n_subjects = len(corrmats_meta["dim1"])
n_subjects_ok, n_conditions, n_perr_sign, n_networks, _ = d.shape

In [ ]:
# Load behavioral model parameters
path_parameter_estimates = join(path["jags"], "parameter_estimates")
alpha = np.load(join(path_parameter_estimates, "alpha_pdci_mle_3digits.npy"))
beta = np.load(join(path_parameter_estimates, "beta_pdci_mle_3digits.npy"))
alpha = alpha[ok_index]
beta = beta[ok_index]

In [ ]:
# Effects of interest
d_interaction = np.mean((d[:, 0] - d[:, 1])[:, 0] - (d[:, 0] - d[:, 1])[:, 1], 
                        axis=0)
d_rew_minus_pun = np.mean(d[:, 0] - d[:, 1], axis=(0, 1))
d_inc_minus_dec = np.mean(d[:, :, 0] - d[:, :, 1], axis=(0, 1))
d_all = np.mean(d, axis=(0, 1, 2)) 

# rmANOVA tstat higher than chance
pvals_mc = np.mean(fstats < fstats_null, axis=0)

# Agreement higher than chance
pvals_d_mc = np.mean(d_all < np.mean(d_null, axis=(1, 2, 3)), axis=0)

pvals_con = pvals_mc[0] + np.tril(pvals_mc[0].T, k=-1)
pvals_perr = pvals_mc[1] + np.tril(pvals_mc[1].T, k=-1)
pvals_interaction = pvals_mc[2] + np.tril(pvals_mc[2].T, k=-1)
pvals_d_mc = pvals_d_mc + np.tril(pvals_d_mc.T, k=-1)

# Multiple comparison correction
pvals_con_fdr = fdrcorrection_matrix(pvals_mc[0])
pvals_perr_fdr = fdrcorrection_matrix(pvals_mc[1])
pvals_interaction_fdr = fdrcorrection_matrix(pvals_mc[2])
pvals_d_mc_fdr = fdrcorrection_matrix(pvals_d_mc)

In [ ]:
plot_matrix(
    d_all,
    labels=network_names,
    annotate=pvals_d_mc,
    annotate_mask=pvals_d_mc_fdr < 0.05,
    title="Network associations"
)

plot_matrix(
    d_rew_minus_pun, 
    clim=(-.15, .15), 
    labels=network_names, 
    annotate=pvals_con, 
    annotate_mask=pvals_con < 0.05,
    title="Condition effect"
)

plot_matrix(
    d_inc_minus_dec, 
    clim=(-.15, .15), 
    labels=network_names, 
    annotate=pvals_perr, 
    annotate_mask=pvals_perr < 0.05,
    title="Prediction error effect"
)

plot_matrix(
    d_interaction, 
    clim=(-.3, .3), 
    labels=network_names, 
    annotate=pvals_interaction_fdr, 
    annotate_mask=pvals_interaction_fdr < 0.05,
    title="Interaction effect"
)

In [ ]:
inc_idx = list(network_names).index("perr_inc")
dec_idx = list(network_names).index("perr_dec")
fp_idx = list(network_names).index("fronto_parietal")
dmn_idx = list(network_names).index("default_mode")
sal_idx = list(network_names).index("salience")
som_idx = list(network_names).index("somatomotor")
q = np.load(join(path_corrmats_unthr_gamma, "q_aggregated.npy"))

In [ ]:
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

In [ ]:
x_rew = np.mean(d[:, :, 0, inc_idx, dec_idx], axis=1)
x_pun = np.mean(d[:, :, 1, inc_idx, dec_idx], axis=1)

y = alpha[:, 1] - alpha[:, 0]  

plt.plot(x_pun, y, 'ro')
plt.xlabel("Integration incPE – decPE")
plt.ylabel(r"$\alpha_+ - \alpha_-$")
plt.title("Punishment")
print(pearsonr(x_pun, y))
plt.show()

plt.plot(x_rew, y, 'ro')
plt.xlabel("Integration incPE – decPE")
plt.ylabel(r"$\alpha_+ - \alpha_-$")
plt.title("Reward")
print(pearsonr(x_rew, y))
plt.show()